# Abgabe 2: Klassische Mechanik 2

**Teammitglieder:** René Zarwel, Michael Wimmer, Matthias Kastenmüller

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from numpy import array, polyval, polyfit
from scipy.optimize import curve_fit

## Aufgabe 1: Umlaufzeit

`<INSERT DESCRIPTION HERE>`

In [ ]:
# Gegebene Funktion & Umgestellt nach r
T = lambda r,C,n: C*r**n
r = lambda T,C,n: (T/C)**(1/n)

# Eingangsdaten
Tdata = array([0.44, 1.61, 3.88, 7.89])
rdata = array([0.088, 0.208, 0.374, 0.600])

plt.loglog(rdata, Tdata, 'o-')

# Logarithmieren der Eingangsdaten
TdataLog = np.log(Tdata)
rdataLog = np.log(rdata)

# Lösen durch Geradengleichung
n = (TdataLog[0]-TdataLog[1])/(rdataLog[0]-rdataLog[1])
C = Tdata[3]/rdata[3]**n
print("C=" + str(C) + " und n=" + str(n))
plt.loglog(rdata, T(rdata,C,n), '*-')

# Berechnung des fehlenden Wertes
Tmiss = 6.20
rmiss = r(6.2,C,n)
print("Für den Wert T=" + str(Tmiss) + " ergibt sich ein r=" + str(rmiss))
plt.loglog(rmiss, Tmiss, '+')

## Aufgabe 6: Drei Sterne

Auf den Stern 1 wirkt die Anziehungskraft von Stern 2 und Stern 3. Zusammen ergeben sie die auf den Stern 1 wirkende Kraft:

$$
\vec{F_1} = m_1 \cdot \ddot{r} = \vec{F_{12}} + \vec{F_{13}}
$$

Damit kann jede wirkende Anziehungskraft separat betrachtet werden. Durch einsetzen der jeweils wirkenden Anziehungskraft ergibt sich folgende Gleichung:

$$
m_1 \cdot \ddot{r} = G\frac{m_2 \cdot m_1}{r_{12}^3}\vec{r_{12}} + G\frac{m_3 \cdot m_1}{r_{13}^3}\vec{r_{13}}
$$

Da $m_1$ auf der rechten Seite ausgeklammert werden kann und somit auf beiden Seiten gleichermaßen vorkommt, kann es aufgehoben werden. Desweiteren ersetzen wir die Richtungsvektoren $\vec{r_{12}}$ und $\vec{r_{13}}$ durch die Berechnung aus den Positionsvektoren. Gleichermaßen wird mit den im Nenner vorhandenen Längen der Richtungsvektoren verfahren. Somit ergibt sich die gegebene Gleichung:

$$
\ddot{\vec{r_1}} = Gm_2\frac{\vec{r_2}-\vec{r_1}}{|\vec{r_2}-\vec{r_1}|^3}+Gm_3\frac{\vec{r_3}-\vec{r_1}}{|\vec{r_3}-\vec{r_1}|^3}
$$

Allgemein kann die Gleichung auch folgendermaßen fomuliert werden:

$$
\ddot{\vec{r_1}} = G \displaystyle\sum_{i=2}^{n} m_i\frac{\vec{r_i}-\vec{r_1}}{|\vec{r_i}-\vec{r_1}|^3} \\
n = \text{ Anzahl der Sterne}
$$

Dies kann auf die anderen Sterne übertragen werden und somit ergeben sich für Stern 2 und 3 folgende Bewegungsgleichungen:

$$
\ddot{\vec{r_2}} = Gm_1\frac{\vec{r_1}-\vec{r_2}}{|\vec{r_1}-\vec{r_2}|^3}+Gm_3\frac{\vec{r_3}-\vec{r_2}}{|\vec{r_3}-\vec{r_2}|^3} \\
\ddot{\vec{r_3}} = Gm_1\frac{\vec{r_1}-\vec{r_3}}{|\vec{r_1}-\vec{r_3}|^3}+Gm_2\frac{\vec{r_2}-\vec{r_3}}{|\vec{r_2}-\vec{r_3}|^3}
$$

In [ ]:
import numpy as np     # get ODE solvers, numpy
import vpython as vp         # get VPython modules for animation
vec=vp.vector

def leapfrog(lfdiffeq, r1, v1, r2, r3, t, h):       # vectorized leapfrog
    """ vector leapfrog method using numpy arrays.
        It solves general (r,v) ODEs as: 
        dr[i]/dt = f[i](v), and dv[i]/dt = g[i](r).
        User supplied lfdiffeq(id, r, v, t) returns
        f[i](r) if id=0, or g[i](v) if id=1.
        It must return a numpy array if i>1 """
    hh = h/2.0
    r11 = r1 + hh*lfdiffeq(0, r1, r2, r3, v1, t)     # 1st: r at h/2 using v0    @\lbl{line:lf1}@
    v11 = v1 +  h*lfdiffeq(1, r11, r2, r3, v1, t+hh)  # 2nd: v1 using a(r) at h/2 @\lbl{line:lf2}@
    r11 = r11 + hh*lfdiffeq(0, r1, r2, r3, v11, t+h)   # 3rd: r1 at h using v1     @\lbl{line:lf3}@
    return r11, v11
  
     
def star(id, r1, r2, r3, v, t):            # return the eqns of motion
    if (id == 0): return v         # velocity, dr/dt
    r12 = r2 - r1
    r13 = r3 - r1
    s12 = vp.mag(vec(r12[0],r12[1],0))   # $s=|\vec{r}|$
    s13 = vp.mag(vec(r13[0],r13[1],0))   # $s=|\vec{r}|$
    return m2*r12/(s12*s12*s12) + m3*r13/(s13*s13*s13)         # accel dv/dt, faster than s**3  
        
def go():
    r1 = np.array([3.0, 1.0]) 
    v1 = np.array([0.0, 0.0]) 
    r2 = np.array([-1.0, -2.0]) 
    v2 = np.array([0.0, 0.0]) 
    r3 = np.array([-1.0, 1.0]) 
    v3 = np.array([0.0, 0.0]) 
    
    # draw the scene, planet earth/path, sun/sunlight               
    scene = vp.canvas(title='Planetary motion',          # scene start 
                       background=vec(.2,.5,1), forward=vec(0,2,-1))
    star1 = vp.sphere(pos=vec(r1[0],r1[1],0), radius=0.1, make_trail=True, up=vec(0,0,1))
    star2 = vp.sphere(pos=vec(r2[0],r2[1],0), radius=0.1, make_trail=True, up=vec(0,0,1))
    star3 = vp.sphere(pos=vec(r3[0],r3[1],0), radius=0.1, make_trail=True, up=vec(0,0,1))
    sunlight = vp.local_light(pos=vec(0,0,0), color=vp.color.yellow) #scn end 
    
    t, h = 0.0, 0.001
    while True:
        vp.rate(200)   # limit animation speed
        r1, v1 = leapfrog(star, r1, v1, r2, r3, t, h)  # integrate 
        star1.pos = vec(r1[0],r1[1],0)           # move planet  
        r2, v2 = leapfrog(star, r2, v2, r1, r3, t, h)  # integrate 
        star2.pos = vec(r2[0],r2[1],0)           # move planet  
        r3, v3 = leapfrog(star, r3, v3, r1, r2, t, h)  # integrate 
        star3.pos = vec(r3[0],r3[1],0)           # move planet  

        
m1 = 150
m2 = 200
m3 = 250
go()